In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'd:\\MLops\\End-to-End-DataScienceProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name:str
    alpha: float
    l1_ratio: float
    target_column: str
    

In [6]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [25]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_model_tarin_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        return model_trainer_config


In [26]:
import pandas as pd
from src.datascience import logger
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib


In [27]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column])
        train_y = train_data[self.config.target_column]
        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio,random_state=42)
        lr.fit(train_x,train_y)
        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))
        logger.info(f"Model is saved at {self.config.root_dir}")
        

In [28]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_tarin_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(e)
    raise


[2025-01-25 14:33:18,215: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2025-01-25 14:33:18,219: INFO: common: yaml file: params.yaml loaded sucessfully]
[2025-01-25 14:33:18,225: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2025-01-25 14:33:18,227: INFO: common: Craeted directory at:artifacts]
[2025-01-25 14:33:19,845: INFO: common: Craeted directory at:artifacts/model_trainer]
[2025-01-25 14:33:19,922: INFO: 2874944050: Model is saved at artifacts/model_trainer]
